In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from neo.core import SpikeTrain
from quantities import ms, s, Hz
from elephant.statistics import mean_firing_rate
from elephant.statistics import time_histogram, instantaneous_rate
from elephant.kernels import GaussianKernel
from elephant.statistics import mean_firing_rate

import os 
import pandas as pd
import seaborn as sns
sns.set_context('talk')
import matplotlib.pyplot as plt

import sys
sys.path.append("G:/My Drive/WORKING_MEMORY/EXPERIMENTS/ELECTROPHYSIOLOGY/ANALYSIS/src/functions/")
import ephys_functions as ephys
import model_functions as mod
import behavioral_functions as beh

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings. filterwarnings('ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

**Check the synch data from the paper**

In [5]:
df = pd.read_csv("G:/My Drive/WORKING_MEMORY/PAPER/ANALYSIS_Figures/instantaneous firing rate_alltrial_5ms.csv", index_col=0)
df_final = pd.read_csv("G:/My Drive/WORKING_MEMORY/PAPER/WM_manuscript_FIGURES/Fig. 7. Synch/synch_data_trials_2beforeSti.csv", index_col=0)

In [ ]:
animal = "E22_2022-01-13_16-34-24.csv"
for animal in df_final.animal.unique():
    # file_name = 'synch_data_trials_2beforeSti'
    # df_final = pd.read_csv(file_name+'.csv', index_col=0)
    threshold = 0.5

    df_session = df_final.loc[df_final.animal==animal]

    fig, ax = plt.subplots(2,1 , figsize=(10,6))
    panel = ax[0]
    panel1 = ax[1]


    sns.lineplot(x="trial", y="synch_window",data=df_session, color='black',ci=68,ax=panel)      
    sns.lineplot(x="trial", y="synch",data=df_session, color='grey',ci=68,ax=panel1)      
    for plot in [panel, panel1]:
        plot.set_ylabel('Synch')
        plot.set_ylim(0.9,max(df_session.synch_window)+0.3)
        plot.fill_between(df_session['trial'],0.9 , 2.5, where=df_session['WM_roll'] <= threshold,
                    facecolor='indigo', alpha=0.3)
        plot.fill_between(df_session['trial'], 0.9, 2.5,  where=df_session['WM_roll'] >= threshold,
                        facecolor='darkgreen', alpha=0.3)
    panel.set_xlabel('Trials')
    panel.set_title('Mouse E22 13-01')
    plt.show()

**Re-do to obtain activity per bin for all neurons**

In [6]:
path = r'E:\Ephys\summary_complete'

In [7]:
# List of variables we want to recover
mean_firing= []
cv=[]
synch = []
repeat_list=[]
hit_list=[]
wm_list=[]
animal_list = []
T_max = []
trial = []
synch_mean_window=[]
aligment_window=[]
surrogates=100

In [ ]:
for filename in os.listdir(os.getcwd()):
    if filename[-3:] != 'pdf':
        df = pd.read_csv(path + "/"+ filename, sep=',',index_col=0)
    else:
        continue
        
    print(filename)   
    
    df = ephys.add_time_before_stimulus(df, 4)

    #Filter for the last 2 seconds of the ITI
    df = df.loc[df.a_END>start]
    
    for align, start, stop, delay_epoch in zip(['END'],[-2],
                                               [0],[False]):

        #Create new aligment to the end of the session
        df['a_'+align] = df.fixed_times-df[align]

        # Sort per trial
        df = df.sort_values('trial')

        #Filter for the last 2 seconds of the ITI
        df = df.loc[(df['a_'+align]>start)&(df['a_'+align]<stop)]

        # total_neurons_session = len(df.loc[(df.trial_start<df.trial)&(df.trial_end>df.trial)].cluster_id.unique())
        total_neurons_session = len(df.cluster_id.unique())
        if total_neurons_session < 30:
            continue
            
        print('There are ' + str(total_neurons_session) + ' neurons')

        for T in df.trial.unique(): 
            # Select the trial that we want to look at this time
            dft = df.loc[df.trial ==T]
                
            dft = dft.loc[(dft['a_'+align]>start)&(dft['a_'+align]<stop)]

            # Recover amount of neurons that were being registered at that trial interval
    #         n_neurons = len(df.loc[(df.trial_start<T)&(df.trial_end>T)].cluster_id.unique())
            n_neurons = len(df.cluster_id.unique())

            # Values for module
            # print("The HB value for this trial is ", dft.accuracy.unique()[0])
            # print("The WM value for this trial is ", dft.repeat_bias.unique()[0])

            if len(dft) <2:
                continue

            hit_list.append(dft.hit.unique()[0])
            repeat_list.append(dft.repeat_choice.unique()[0])
            wm_list.append(dft.WM_roll.unique()[0])

            if delay_epoch:
                # if dft.delay.unique()[0] == 0.1 or dft.delay.unique()[0] == 0.2:
                if dft.delay.unique()[0] != 1 and dft.delay.unique()[0] == 3 and dft.delay.unique()[0] == 3:
                    mean_firing.append(np.nan)
                    synch.append(np.nan)
                    trial.append(T)
                    animal_list.append(filename)
                    aligment_window.append(align+'_'+str(start)+'_'+str(stop))
                    T_max.append(max(df.trial.unique()))
                    continue
                    
            times_spikes = dft['a_'+align].values
            times_spikes = times_spikes*1000*ms #transform to ms

            ############################################################ Set the strat and end time of the train
            stop_time =  stop*1000*ms ## End of the trial in ms
            start_time = start*1000*ms ## Start of the trial in ms     

            ############################################################ Spiketrain
            spiketrain = SpikeTrain(times_spikes, units=ms, t_stop=stop_time, t_start=start_time) 

            ############################################################ 
            histogram_rate = time_histogram([spiketrain], 20*ms, output='counts')
            times_ = histogram_rate.times.rescale(s)
            firing_real = histogram_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()

            mean_firing.append(float(mean_firing_rate(spiketrain)/n_neurons*1000))
            print("The mean firing rate of spiketrain is", mean_firing_rate(spiketrain)/n_neurons)

            print("The std of the firing rate of spiketrain is", np.std(firing))
            real_std=np.std(firing_real) # Store the real std value

            # t1_start = process_time() 
            list_std = []
            for i in range(surrogates):
                # Create a random shuffle for the same amount of spikes in that interval
                random_float_list = np.random.uniform(start, stop, len(times_spikes))
                # t1_stop = process_time() 
                # print(str(T) + ' ' + str(t1_stop-t1_start))
                
                surrogate_spikes = np.array(random_float_list)*1000*ms #transform to ms
                spiketrain = SpikeTrain(surrogate_spikes, units=ms, t_stop=stop_time, t_start=start_time) 

                histogram_rate = time_histogram([spiketrain], 20*ms, output='rate')
                times_ = histogram_rate.times.rescale(s)
                firing = histogram_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()

                list_std.append(np.std(firing))

            synch.append(real_std/np.mean(list_std))
            
            trial.append(T)
            animal_list.append(filename)
            aligment_window.append(align+'_'+str(start)+'_'+str(stop))
            T_max.append(max(df.trial.unique()))

            # Organized by cluster_id and corrected for FR ____________________________
#             cluster_id=[]
#             FR_mean=[]

#             for N in df.cluster_id.unique():
#                 spikes = dft.loc[dft.cluster_id==N]['a_'+align].values
#                 FR_mean.append(len(spikes)/abs(stop-start))
#                 cluster_id.append(N)

#             df_spikes = pd.DataFrame(list(zip(cluster_id,FR_mean)), columns =['cluster_id','FR'])
#             df_spikes = df_spikes.sort_values('FR')
#             df_spikes['new_order'] = np.arange(len(df_spikes))

#             dft = pd.merge(df_spikes, dft, on=['cluster_id'])

#             print('Synch:', real_std/np.mean(list_std), '; WM:', str(dft.WM_roll.unique()[0]))

#             fig, (ax,ax1)= plt.subplots(2, 1,figsize=(10, 6), sharex=True)

#             j=0
#             for N in dft.new_order.unique():
#                 spikes = dft.loc[dft.new_order==N]['a_'+align].values
#                 j+=1
#                 ax.plot(spikes,np.repeat(j, len(spikes)), '|', markersize=5, color='black', zorder=1)

#             # sns.scatterplot(x='a_'+align,y='cluster_id',data=dft, ax=ax, size=1, legend=False, marker='|') # Plot neuron per neuron, not corrected by cluster_id
#             # ax.eventplot([spiketrain], linelengths=0.75, color='black') # Plot all neurons together in a spiketrain

#             ax.set_title('Trial: ' + str(T))
#             ax1.plot(times_,firing_real/n_neurons*1000)
#             ax1.set_xlabel('Time (s) from end of the trial', fontsize=16)
#             sns.despine()
#             plt.show()

        # Create new dataframe with all the recovered lists. 
        # df_final = pd.DataFrame(list(zip(synch, np.arange(len(synch)))),
        #        columns =['synch','trial'])
        # synch_window= compute_window_centered(df_final,5,'synch')
        # synch_mean_window.extend(synch_window)

In [ ]:
# Create new dataframe with all the recovered lists. 
df_final = pd.DataFrame(list(zip(mean_firing, synch, hit_list, wm_list, repeat_list, T_max,trial,animal_list, aligment_window)), 
       columns =['FR', 'synch','hit','WM_roll','repeat','total trials','trial','animal', 'aligment'])

df_final.dropna(inplace=True)

In [ ]:
df_final['accuracy'] = compute_window_centered(df_final,10,'hit')
df_final['repeat_bias'] = compute_window_centered(df_final,10,'repeat')
df_final['synch_window']= compute_window_centered(df_final,5,'synch')

In [ ]:
def trials(row):
    val = 0
    val = row['trial']/row['total trials']
    return val

df_final['T_norm'] = df_final.apply(trials, axis=1) 
df_final['z_synch_window'] = stats.zscore(df_final['synch_window'].values, nan_policy='omit')
df_final['z_synch'] = stats.zscore(df_final['synch'].values, nan_policy='omit')
df_final['z_FR'] = stats.zscore(df_final['FR'].values, nan_policy='omit')
df_final['state'] = np.where(df_final['WM_roll'] >0.6, 1, 0)

In [ ]:
save_path = 'C:/Users/Tiffany/Google Drive/WORKING_MEMORY/PAPER/Figures/'
os.chdir(save_path)
file_name = 'synch_data'
df_final.to_csv(file_name+'.csv')

**Measure activity of a specific bin for the rank analysis**

In [1]:
def interval_extraction_trial(df, cluster_list=[], variable = 'vector_answer', align = 'Delay_OFF', start = 0, stop = 1, delay_only=False):
    y = []
    d = {}
    
    if delay_only == False:
        # print('Skipping delays')
        if align == 'Delay_OFF' and start < 0:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2)]
        if align == 'Delay_OFF' and start < -1:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2) & (df.delay != 1)]

        if align == 'Stimulus_ON' and stop > 0.5:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2)]

        if align == 'Stimulus_ON' and stop > 1.5:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2) & (df.delay != 1)]
    
    # print('Recovered from: ', str(len(df.trial.unique())), ' trials')
    # Create new aligment to the end of the session
    df['a_'+align] = df.fixed_times-df[align]

    # cluster_list = df_all.cluster_id.unique()
    df = df.sort_values('trial')
    
    y = df.groupby('trial')[variable].mean()

    # Filter for the spikes that occur in the interval we are analyzing
    df = df.loc[(df['a_'+align]>start)&(df['a_'+align]<stop)]

    df_final = pd.DataFrame()
    df_final = df.groupby(['trial','cluster_id']).count()
    df_final.reset_index(inplace=True)
    df_final = df_final.pivot_table(index=['trial'], columns='cluster_id', values='fixed_times', fill_value=0).rename_axis(None, axis=1)
    df_final = df_final.reindex(cluster_list, axis=1,fill_value=0)

    result = pd.merge(df_final, y, how="right", on=["trial"]).fillna(0)
    result = result.rename(columns={variable: "y"})
    result['y'] = np.where(result['y'] == 0, -1, result['y']) 
    
    return result, result['y']

In [ ]:
for filename in os.listdir(os.getcwd()):
    if filename[-3:] != 'pdf':
        df = pd.read_csv(path + "/"+ filename, sep=',',index_col=0)
    else:
        continue
        
    print(filename)   
    
    df = ephys.add_time_before_stimulus(df, 4)
    df = df.sort_values('trial')

    df['delay'] = np.around(df.delay,2)
    df['state'] = np.where(df['WM_roll'] >0.6, 1, 0)
    
    decode = 'vector_answer'
    align='Stimulus_ON'
    start = -2
    stop = 0
    cluster_list = df.cluster_id.unique()
    for T in df.trial.unique():
        for start_, stop_ in np.arange(start, stop, 0.020):
            print(T)
            df_final, y = interval_extraction_trial(df, variable = decode, align = align, start = start, stop = stop, cluster_list=cluster_list)
            df_final.reset_index(inplace=True)
            df_final = df_final.drop(columns ='trial')
            df_final['state'] = df['state']
            df_final['trial'] = T
            df_final['session'] = filename
            df_final['times'] =  str(start_) + '_' + str(stop_)
            
            # session, trial, times, state, cluster_id, count